<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    npr Mini-Challenge 1: <br>TFIDF-HGBC
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Florin Barbisch, Gabriel Torres Gamez und Jan Zwicky im HS 2023.
</p>

## Vorgegebenes Ziel

Wir evaluieren 3 verschiedene Klassifikationssysteme in einer Textklassifikationsaufgabe. <br><br>
Wir argumentieren, warum wir diese Systeme für diesen Fall ausgewählt haben, und bauen, trainieren und bewerten das System. <br><br>
Wir führen auch eine gründliche Fehleranalyse durch und schlagen Theorien zur Verbesserung der Systeme vor, <br>
insbesondere durch die Analyse einzelner Fälle und den Vergleich der Vorhersagen jedes Systems. <br>
Mindestens eines der Systeme basiert auf einem feinabgestimmten Deep-Learning-Modell.

Hier können wir Daten aus gemeinsamen Aufgaben verwenden, wie zum Beispiel Semeval, Kaggle, <br>Codalab oder Paperwithcode für die Textklassifikation.

## Requirements, Imports und Einstellungen
Hier werden die benötigten Python-Pakete importiert und die Einstellungen für die Plots
vorgenommen. 

In [1]:
# All Imports
import sys
import torch
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split

# Versions of the packages used
print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Matplotlib Version: {plt.matplotlib.__version__}")
print(f"Sklearn Version: {sklearn.__version__}")
print(f"Seaborn Version: {sns.__version__}")

# Warnings Settings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Numpy Settings
np.set_printoptions(precision=2, suppress=True)
np.random.seed(42)

# Matplotlib Settings
plt.rcParams["figure.figsize"] = (24, 12)

Python Version: 3.10.13 (main, Aug 24 2023, 12:59:26) [Clang 15.0.0 (clang-1500.0.40.1)]
PyTorch Version: 2.0.1
Numpy Version: 1.23.5
Pandas Version: 2.1.1
Matplotlib Version: 3.6.3
Sklearn Version: 1.3.1
Seaborn Version: 0.12.2


## Helper Functions

Hier werden Funktionen definiert, die bei beiden Modellen verwendet werden.

In [2]:
# Metrics, shortcuts, constants, etc.

## Gewählter Datensatz

Wir haben uns für den <a href="https://www.kaggle.com/competitions/nlp-getting-started/data">"Natural Language Processing with Disaster Tweets"</a> Datensatz entschieden.<br><br>
Der Datensatz passt zum Ziel der Mini Challenge, da er Textdaten aus Twitter enthält, die in Notfällen <br>als wichtiger Kommunikationskanal dienen. Dies ermöglicht die Entwicklung und Evaluierung von <br>Textklassifikationssystemen, um Tweets zu identifizieren, die echte Katastrophen ankündigen, <br>von solchen, die dies nicht tun. <br><br>Dies erfordert eine gründliche Fehleranalyse und die Entwicklung von Modellen, die in der Lage sind, <br>die Bedeutung von Wörtern und Ausdrücken im Kontext zu verstehen, da Tweets oft metaphorische <br>Sprache verwenden können. <br><br>Wir haben Zugriff auf einen Datensatz von 10.000 handklassifizierten Tweets, der eine solide <br>Grundlage für die Evaluierung und Verbesserung von Textklassifikationssystemen bietet.

## Einlesen des Verarbeiteten Datensatzes

## Klassifikationsmodell

[BESCHREIBUNG DES MODELLS]

### Auswertung des Modells

## Evaluation
In der Evaluation beschreiben wir, welche Metrik verwendet wurde, wieso die Metrik für den Anwendungsfall passt und diskutieren die Ergebnisse der Experimente und einige Vorhersagen auf einzelnen Testsamples.

## Erkenntnisse

## Fazit